# Collate iNaturalist taxon status data

Taxa in iNaturalist have conservation statuses that the ALA is responsible for maintaining. The process for bulk loads is to submit the data to iNaturalist in December/January using these [templates and checklists](https://docs.google.com/spreadsheets/d/1yTwWh4d-lHeaBGCB9m70-HKEMtvrquHsPu3Zrgz9BcE/edit#gid=1531097917)

Current statuses per iNaturalist taxonID are available in the iNaturalist site export, accessible via an iNaturalist AU site admin and in this repository (inaturalist-australia-9-conservation_statuses.xls)

**Steps covered in this notebook:**
1. Extract all the current statuses in Australian jurisdictions in the export (eg.AU or Australia place, .gov.au in the URL or user 708886)
2. Resolve the taxon name for each using the iNaturalist Taxon DwCA at https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip (~350MB) or the iNat taxon API (if the taxon is inactive in iNaturalist)

File output: `inat-aust-status-taxa.csv` containing Australian conservation statuses,

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (ADD)
    b. updates - any changes to statuses (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

In [1]:
# setup
import pandas as pd
# projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/inaturalist-statuses"

## Read in the iNaturalist Conservation Statuses export
Find the Australian statuses by querying:
* place names and display place names containing `AU`, `Australia` etc
* urls containing the string `.gov.au`
* records with a user id 708886 (Peggy who submitted the last round of statuses)

In [2]:
df = pd.read_csv(sourcedir + "inaturalist-australia-9-conservation_statuses.csv", encoding='UTF-8', na_filter=False, dtype=str)
df

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,iucn,created_at,updated_at,place_name,place_display_name
0,210936,78570,,,,IUCN Red List,LC,https://www.iucnredlist.org/species/109716665/...,,,10,2021-09-28 11:12:51.830044,2021-12-21 04:53:02.568719,,
1,228106,219800,119123,6883,,NatureServe,S2,https://explorer.natureserve.org/,,,40,2021-12-16 19:59:11.692913,2021-12-21 13:14:46.38026,Ontario,"Ontario, CA"
2,224080,239472,51061,50,,NatureServe,S2S3,https://explorer.natureserve.org/Taxon/ELEMENT...,,,0,2021-11-24 18:14:30.91353,2021-11-24 18:14:30.91353,Nevada,"Nevada, US"
3,228110,219828,119123,6834,,NatureServe,S2S4,https://explorer.natureserve.org/,,,30,2021-12-16 19:59:12.175319,2021-12-21 13:18:11.648863,Alberta,"Alberta, CA"
4,223126,155851,,6815,,Tatzpiteva,VU,http://tatzpiteva.org.il/,,obscured,30,2021-10-03 01:25:45.177631,2021-10-03 01:25:45.177631,Israel,Israel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253484,268980,1430390,,,,IUCN Red List,LC,https://www.iucnredlist.org/species/22725894/1...,,open,10,2022-12-02 18:17:39.897104,2022-12-02 18:17:39.897104,,
253485,268981,1430381,,,,IUCN Red List,LC,https://www.iucnredlist.org/species/22725888/1...,,open,10,2022-12-02 18:18:40.547528,2022-12-02 18:18:40.547528,,
253486,268982,1425294,1368063,7190,,Ministerio de Ambiente y Desarrollo Sustentabl...,IC,https://avesargentinas.org.ar/sites/default/fi...,Insuficientemente conocido,open,5,2022-12-02 18:47:31.444653,2022-12-02 18:47:31.444653,Argentina,Argentina
253487,268983,926017,28,,,San Diego Society of Natural History,Threatened,https://sdplantatlas.org/pdffiles/BajaChecklis...,"Plants rare, threatened, or endangered in Cali...",obscured,30,2022-12-02 21:33:56.076056,2022-12-02 21:33:56.076056,,


In [4]:
# list of unique Aust place display names
placedisplaydf = df['place_display_name'].drop_duplicates().sort_values()
placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains("AU|Australia|AUSTRALIA")]
placedisplaydf

778                               Australia
84785     Australia Exclusive Economic Zone
2716       Australian Capital Territory, AU
129631                Cairns - Pt B, QL, AU
173650         Great Barrier Island, AU, NZ
136230         Lower Eyre Peninsula, SA, AU
457                     New South Wales, AU
352                  Northern Territory, AU
381                          Queensland, AU
175360                  Rottnest Island, AU
3751                    South Australia, AU
174509       South Australia, marine waters
247882        South East Queensland, QL, AU
4878                           Tasmania, AU
248                            Victoria, AU
982                   Western Australia, AU
129741                     Yarrabah, QL, AU
Name: place_display_name, dtype: object

In [5]:
# list of unique Aust place names
placedf = df['place_name'].drop_duplicates().sort_values()
placedf = placedf[pd.Series(placedf).str.contains("AU|Australia|AUSTRALIA")]
placedf

778                               Australia
84785     Australia Exclusive Economic Zone
2716           Australian Capital Territory
3751                        South Australia
174509       South Australia, marine waters
982                       Western Australia
Name: place_name, dtype: object

In [6]:
# list of unique Aust Govt urls
urldf = df['url'].drop_duplicates().sort_values()
urldf = urldf[pd.Series(urldf).str.contains(".gov.au")]
urldf

205924     https://apps.des.qld.gov.au/species-search/de...
248380     https://www.environment.gov.au/epbc/about/epb...
5803      http://environment.gov.au/cgi-bin/sprat/public...
213823    http://environment.gov.au/cgi-bin/sprat/public...
2431      http://www.environment.gov.au/biodiversity/thr...
                                ...                        
173380    https://www.qld.gov.au/environment/plants-anim...
247223    https://www.threatenedspecieslink.tas.gov.au/P...
175392    https://www.threatenedspecieslink.tas.gov.au/P...
5905      https://www.threatenedspecieslink.tas.gov.au/P...
140039    https://www.threatenedspecieslink.tas.gov.au/P...
Name: url, Length: 672, dtype: object

In [7]:
# filter out all of these concat all, along with any records created by us (user id=708886) and remove duplicates
dfaus = pd.concat([df.apply(lambda row: row[df['place_display_name'].isin(placedisplaydf)]),
                   df.apply(lambda row: row[df['place_name'].isin(placedf)]),
                   df.apply(lambda row: row[df['url'].isin(urldf)]),
                   df.apply(lambda row: row[df['user_id'] == '708886'])]).drop_duplicates()
dfaus.sort_values(['taxon_id','user_id'])

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,iucn,created_at,updated_at,place_name,place_display_name
106342,268186,100080,1138587,6744,,,Not listed,,,open,10,2022-10-25 05:20:17.532504,2022-10-25 05:20:17.532504,Australia,Australia
100874,268138,100118,1138587,6744,,,Not listed,,,open,10,2022-10-25 03:32:38.344862,2022-10-25 03:32:38.344862,Australia,Australia
104815,268168,100127,1138587,6744,,,Not listed,,,open,10,2022-10-25 03:58:15.923985,2022-10-25 03:58:15.923985,Australia,Australia
96905,223098,1003032,3669610,7308,,Queensland Nature Conservation Act 1992,CE,https://apps.des.qld.gov.au/species-search/det...,,obscured,50,2021-10-01 10:37:36.857571,2022-02-09 09:34:29.493507,Queensland,"Queensland, AU"
232091,223099,1003032,3669610,6744,,Environment Protection and Biodiversity Conser...,CE,https://www.environment.gov.au/cgi-bin/sprat/p...,,obscured,50,2021-10-01 10:37:36.919705,2022-07-11 09:27:11.063411,Australia,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244087,153828,99970,708886,7830,16656,Victoria Flora and Fauna Guarantee Act 1988,Critically Endangered,https://www.environment.vic.gov.au/conserving-...,,obscured,50,2019-07-23 00:13:02.234163,2022-06-10 15:22:49.67085,Victoria,"Victoria, AU"
5576,264607,99971,3249428,7830,,Victoria Flora and Fauna Guarantee Act 1988,Endangered,https://www.environment.vic.gov.au/conserving-...,,obscured,40,2022-06-06 17:00:58.905798,2022-06-06 17:04:18.766421,Victoria,"Victoria, AU"
238419,153613,99973,708886,6827,16654,WA Department of Environment and Convservation,critically endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,50,2019-07-23 00:12:11.19598,2019-07-23 00:12:11.19598,Western Australia,"Western Australia, AU"
238550,153742,99974,708886,6827,16654,WA Department of Environment and Convservation,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,40,2019-07-23 00:12:41.014669,2019-07-23 00:12:41.014669,Western Australia,"Western Australia, AU"


## Retrieve taxon info from iNaturalist
The above file contains only the taxon identifier. Retrieve the full taxon name and classifications.

In [8]:
%%script echo skipping # comment this line to download dataset from the web and save locally - add inaturalist-taxonomy.dwca.zip to .gitignore

# save the file to the source data directory
import requests

url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

r = requests.get(url)
with open(sourcedir + filename, 'wb') as f:
    f.write(r.content)
# reminder: add inaturalist-taxonomy.dwca.zip to .gitignore

skipping # comment this line to download dataset from the web and save locally - add inaturalist-taxonomy.dwca.zip to .gitignore


In [9]:
# open the file in the source data directory and read the taxa.csv file
import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive)
z.close()

inattaxa.head(10)

,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...
3,4,https://www.inaturalist.org/taxa/4,https://www.inaturalist.org/taxa/4,https://www.inaturalist.org/taxa/3,Animalia,Chordata,Aves,Gruiformes,NaN,NaN,NaN,NaN,2019-10-19T15:14:18Z,Gruiformes,order,http://www.catalogueoflife.org/annual-checklis...
4,5,https://www.inaturalist.org/taxa/5,https://www.inaturalist.org/taxa/5,https://www.inaturalist.org/taxa/4,Animalia,Chordata,Aves,Gruiformes,Aramidae,NaN,NaN,NaN,2022-03-24T16:38:28Z,Aramidae,family,http://www.birdlife.org/datazone/speciessearch...
5,6,https://www.inaturalist.org/taxa/6,https://www.inaturalist.org/taxa/6,https://www.inaturalist.org/taxa/5,Animalia,Chordata,Aves,Gruiformes,Aramidae,Aramus,NaN,NaN,2020-02-11T06:43:19Z,Aramus,genus,http://www.birdlife.org/datazone/speciessearch...
6,7,https://www.inaturalist.org/taxa/7,https://www.inaturalist.org/taxa/7,https://www.inaturalist.org/taxa/6,Animalia,Chordata,Aves,Gruiformes,Aramidae,Aramus,guarauna,NaN,2022-03-30T18:35:55Z,Aramus guarauna,species,http://www.birdlife.org/datazone/speciesfactsh...
7,12,https://www.inaturalist.org/taxa/12,https://www.inaturalist.org/taxa/12,https://www.inaturalist.org/taxa/71262,Animalia,Chordata,Aves,Cariamiformes,Cariamidae,NaN,NaN,NaN,2022-03-24T16:37:54Z,Cariamidae,family,http://www.birdlife.org/datazone/speciessearch...
8,13,https://www.inaturalist.org/taxa/13,https://www.inaturalist.org/taxa/13,https://www.inaturalist.org/taxa/12,Animalia,Chordata,Aves,Cariamiformes,Cariamidae,Cariama,NaN,NaN,2018-12-19T08:58:24Z,Cariama,genus,http://www.birdlife.org/datazone/speciessearch...
9,14,https://www.inaturalist.org/taxa/14,https://www.inaturalist.org/taxa/14,https://www.inaturalist.org/taxa/13,Animalia,Chordata,Aves,Cariamiformes,Cariamidae,Cariama,cristata,NaN,2021-07-06T02:04:43Z,Cariama cristata,species,http://www.birdlife.org/datazone/speciesfactsh...


In [12]:
len(inattaxa) # it's quite big

1216979

In [13]:
# left join to filter just the taxon that have statuses that we're interested in
austtaxaids = dfaus['taxon_id'].drop_duplicates()
inattaxa['id'] = inattaxa['id'].astype(str)
inataustaxa = pd.merge(austtaxaids, inattaxa, how="left", left_on='taxon_id', right_on='id')
inataustaxa.sort_values('id')

,taxon_id,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
477,100080,100080,https://www.inaturalist.org/taxa/100080,https://www.inaturalist.org/taxa/100080,https://www.inaturalist.org/taxa/49497,Animalia,Chordata,Actinopterygii,Perciformes,Epinephelidae,Epinephelus,bleekeri,NaN,2019-11-24T03:20:37Z,Epinephelus bleekeri,species,http://www.fishbase.org
437,100118,100118,https://www.inaturalist.org/taxa/100118,https://www.inaturalist.org/taxa/100118,https://www.inaturalist.org/taxa/49497,Animalia,Chordata,Actinopterygii,Perciformes,Epinephelidae,Epinephelus,malabaricus,NaN,2019-11-23T03:14:55Z,Epinephelus malabaricus,species,http://www.fishbase.org
462,100127,100127,https://www.inaturalist.org/taxa/100127,https://www.inaturalist.org/taxa/100127,https://www.inaturalist.org/taxa/49497,Animalia,Chordata,Actinopterygii,Perciformes,Epinephelidae,Epinephelus,polyphekadion,NaN,2019-02-06T23:51:50Z,Epinephelus polyphekadion,species,http://www.fishbase.org
431,1003032,1003032,https://www.inaturalist.org/taxa/1003032,https://www.inaturalist.org/taxa/1003032,https://www.inaturalist.org/taxa/142613,Plantae,Tracheophyta,Polypodiopsida,Polypodiales,Pteridaceae,Antrophyum,austroqueenslandicum,NaN,2021-10-01T10:37:36Z,Antrophyum austroqueenslandicum,species,NaN
102,1004434,1004434,https://www.inaturalist.org/taxa/1004434,https://www.inaturalist.org/taxa/1004434,https://www.inaturalist.org/taxa/140567,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Oberonia,crateriformis,NaN,2021-07-18T09:05:24Z,Oberonia crateriformis,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,1091300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2335,538090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,1038965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2617,733329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Retrieve inactive taxa names from API

In [14]:
# looking at those that didn't match in the left join - Inactive or problem taxon - need to go to the API to get the detail
unmatchedtaxonids = inataustaxa[inataustaxa['id'].isna()]['taxon_id']
unmatchedtaxonids

11      1262199
113     1264437
132     1264442
187      952472
293     1202976
         ...   
2333    1091300
2335     538090
2503    1038965
2617     733329
3040      50744
Name: taxon_id, Length: 81, dtype: object

In [15]:
%%script echo skipping # comment this line to download dataset from the web and save locally - add inaturalist-taxonomy.dwca.zip to .gitignore

# to go the API to retrieve taxon names
import requests
import json
from time import sleep

apiurlbase = "https://api.inaturalist.org/v1/taxa/"
taxonlist = []
i = 1
for unmatchedid in unmatchedtaxonids:
    print(str(i) + " " + unmatchedid)
    apiurl = apiurlbase + str(unmatchedid)
    response = requests.request("GET", apiurl)
    payload = json.loads(response.text)
    if (payload['total_results'] == 1):
        r = payload['results'][0]
        try:
            common_name = r['preferred_common_name']
        except KeyError:
            common_name = ""
        taxonlist.append({'taxon_id': unmatchedid,
                      'name':r['name'],
                      'preferred_common_name':common_name,
                      'is_active':r['is_active'],
                      'observation_count':r['observations_count'],'current_synonymous_taxon_ids':r['current_synonymous_taxon_ids']})
    else:
        print("Warning: taxon_id " + unmatchedid + "returns more than one result from inaturalist")
    sleep(1)
    i+=1

pd.DataFrame(taxonlist).to_csv(sourcedir + "inat-aust-inactive-taxon.csv",index = False)

skipping # comment this line to download dataset from the web and save locally - add inaturalist-taxonomy.dwca.zip to .gitignore


In [16]:
inactivetaxa = pd.read_csv(sourcedir+"inat-aust-inactive-taxon.csv", dtype=str)
inactivetaxa

,taxon_id,name,preferred_common_name,is_active,observation_count,current_synonymous_taxon_ids
0,1262199,Caleana dixonii,Sandplain Duck Orchid,False,0,[]
1,1264437,Arctophoca forsteri,New Zealand Fur Seal,False,0,[41752]
2,1264442,Arctophoca tropicalis,Subantarctic Fur Seal,False,0,[41753]
3,952472,Blechnum deltoides,NaN,False,0,[1348449]
4,1202976,Petauroides armillatus,Central Greater Glider,False,0,[42783]
...,...,...,...,...,...,...
76,1091300,Plectranthus habrophyllus,NaN,False,0,[1278873]
77,538090,Pararistolochia praevenosa,NaN,False,0,[866782]
78,1038965,Boronia anemonifolia variabilis,NaN,False,0,[1426173]
79,733329,Leucopogon woodsii,Broom Beard-heath,False,0,[1399666]


The taxa above are typically marked as inactive, which means they have no observations in iNaturalist. Collate them into the taxa list, and merge the name into the scientificName field for later use.


In [17]:
# collate the status and the taxon info into a single file to use for the state work
# scientificName is 1. if id field is null, the name field or 2. else it's scientificName
alltaxa = pd.merge(inataustaxa, inactivetaxa, how="left")
alltaxa['scientificName'] = alltaxa.apply(lambda x: x['name'] if pd.isnull(x['id']) else x['scientificName'],axis=1)
alltaxa = alltaxa.drop(['name','observation_count'],axis=1)
alltaxa


,taxon_id,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,38493,38493,https://www.inaturalist.org/taxa/38493,https://www.inaturalist.org/taxa/38493,https://www.inaturalist.org/taxa/38479,Animalia,Chordata,Reptilia,Squamata,Scincidae,Eulamprus,kosciuskoi,NaN,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,NaN,NaN,NaN
1,918383,918383,https://www.inaturalist.org/taxa/918383,https://www.inaturalist.org/taxa/918383,https://www.inaturalist.org/taxa/430819,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Chiloschista,phyllorhiza,NaN,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,NaN,NaN,NaN
2,1033183,1033183,https://www.inaturalist.org/taxa/1033183,https://www.inaturalist.org/taxa/1033183,https://www.inaturalist.org/taxa/624221,Plantae,Tracheophyta,Magnoliopsida,Proteales,Proteaceae,Eidothea,hardeniana,NaN,2021-02-22T07:21:17Z,Eidothea hardeniana,species,NaN,NaN,NaN,NaN
3,1247288,1247288,https://www.inaturalist.org/taxa/1247288,https://www.inaturalist.org/taxa/1247288,https://www.inaturalist.org/taxa/184926,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rhamnaceae,Pomaderris,bodalla,NaN,2021-08-27T06:18:35Z,Pomaderris bodalla,species,https://eol.org/pages/49432063,NaN,NaN,NaN
4,1255510,1255510,https://www.inaturalist.org/taxa/1255510,https://www.inaturalist.org/taxa/1255510,https://www.inaturalist.org/taxa/321172,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Leionema,gracile,NaN,2021-10-18T22:35:58Z,Leionema gracile,species,https://eol.org/pages/47126412,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,1363122,1363122,https://www.inaturalist.org/taxa/1363122,https://www.inaturalist.org/taxa/1363122,https://www.inaturalist.org/taxa/1069166,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Lamiaceae,Coleus,leiperi,NaN,2022-03-05T05:49:07Z,Coleus leiperi,species,https://powo.science.kew.org/taxon/urn:lsid:ip...,NaN,NaN,NaN
3039,1182117,1182117,https://www.inaturalist.org/taxa/1182117,https://www.inaturalist.org/taxa/1182117,https://www.inaturalist.org/taxa/142081,Plantae,Tracheophyta,Magnoliopsida,Caryophyllales,Amaranthaceae,Achyranthes,margaretarum,NaN,2021-02-12T19:39:39Z,Achyranthes margaretarum,species,http://plantsoftheworldonline.org/taxon/urn:ls...,NaN,NaN,NaN
3040,50744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stiphodon allen,NaN,NaN,Opal Cling-Goby,False,[]
3041,924263,924263,https://www.inaturalist.org/taxa/924263,https://www.inaturalist.org/taxa/924263,https://www.inaturalist.org/taxa/576828,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Reedia,spathacea,NaN,2020-09-27T12:27:42Z,Reedia spathacea,species,NaN,NaN,NaN,NaN


In [18]:
# now merge back with the original australian statuses and save locally
taxastatus = pd.merge(dfaus,alltaxa,how="left", on="taxon_id")
#taxastatus
#taxastatus.drop(['id_y'])
taxastatus = taxastatus.rename(columns={'id_x':'id'})
taxastatus

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,NaN,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,NaN,NaN,NaN
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,NaN,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,NaN,NaN,NaN
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,NaN,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,NaN,NaN,NaN
3,166416,1033183,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,obscured,...,Eidothea,hardeniana,NaN,2021-02-22T07:21:17Z,Eidothea hardeniana,species,NaN,NaN,NaN,NaN
4,180721,1247288,222137,6825,,NSW Threatened Species Scientific Committee,vu,https://www.environment.nsw.gov.au/topics/anim...,,obscured,...,Pomaderris,bodalla,NaN,2021-08-27T06:18:35Z,Pomaderris bodalla,species,https://eol.org/pages/49432063,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,165697,1182117,3669610,73684,,Australian Government,CR,http://www.environment.gov.au/biodiversity/thr...,,,...,Achyranthes,margaretarum,NaN,2021-02-12T19:39:39Z,Achyranthes margaretarum,species,http://plantsoftheworldonline.org/taxon/urn:ls...,NaN,NaN,NaN
3449,130966,508985,,,,,critically endangered,http://environment.gov.au/cgi-bin/sprat/public...,,obscured,...,Lichenostomus,melanops,cassidix,2022-06-11T03:51:49Z,Lichenostomus melanops cassidix,subspecies,http://www.birds.cornell.edu/clementschecklist...,NaN,NaN,NaN
3450,161226,50744,516268,,,DAWE Species Profile and Threats Database,CR,https://www.environment.gov.au/cgi-bin/sprat/p...,,obscured,...,NaN,NaN,NaN,NaN,Stiphodon allen,NaN,NaN,Opal Cling-Goby,False,[]
3451,162783,924263,764897,,,"Department of Biodiversity, Conservation and A...",EN,https://www.dpaw.wa.gov.au/images/documents/pl...,,obscured,...,Reedia,spathacea,NaN,2020-09-27T12:27:42Z,Reedia spathacea,species,NaN,NaN,NaN,NaN


In [19]:
taxastatus.to_csv(sourcedir + "inat-aust-status-taxa.csv", index=False)

In [20]:
taxastatus.groupby(['taxonRank'])['taxonRank'].count()

taxonRank
complex          2
form             1
genus           42
hybrid           4
order            1
species       3239
subspecies      68
variety         11
Name: taxonRank, dtype: int64

## Notes about conservation status inheritance in inaturalist:
Adding a conservation status for a higher level taxon affects observations of all the species in this taxon. Please do not add statuses for taxa that contain species that have no status because that will incorrectly obscure coordinates for observations of those species.

### Example:
Genus Acriopsis - https://www.inaturalist.org/taxa/425476-Acriopsis

The inherited species records are not in the export and can't be edited, but they do appear on the species pages.
The changes are to set _Acriopsis emarginata_ to Vulnerable. The rest will remain least concern.

In [21]:
eg = pd.DataFrame(columns=('TaxonID', 'Name', 'Status','Rank','iNat species page','iNat edit taxon','Export'))
eg.loc[1]=['425476','Acriopsis','LC/obscured','genus','No ','Yes','Yes']
eg.loc[2]=['1141144','Acriopsis emarginata','LC/obscured','species','Yes','No','No']
eg.loc[3]=['425475','Acriopsis liliifolia','LC/obscured','species','Yes','No','No']
eg.loc[4]=['427833','Acriopsis ridleyi','LC/obscured','species','Yes','No','No']
eg.loc[5]=['1037999','Acriopsis indica','LC/obscured','Species','Yes','No','No']
eg

,TaxonID,Name,Status,Rank,iNat species page,iNat edit taxon,Export
1,425476,Acriopsis,LC/obscured,genus,No,Yes,Yes
2,1141144,Acriopsis emarginata,LC/obscured,species,Yes,No,No
3,425475,Acriopsis liliifolia,LC/obscured,species,Yes,No,No
4,427833,Acriopsis ridleyi,LC/obscured,species,Yes,No,No
5,1037999,Acriopsis indica,LC/obscured,Species,Yes,No,No
